## br - Binary Resolver and rp - Recursive Partitions

# Datasets

The results for this run are for the *xroadsc* dataset: this does not use the very partial road condition data.

```
impute0.dat

ppl00.dat
ppl0.dat

xroadsc-3.dat
xroadsc-4.dat
xroadsc-5.dat
xroadsc-7.dat
xroadsc-9.dat
xroadsc-cwy0.dat
```

# Data flow

1. br00.R loads the CSV and saves to ppl00.dat
2. br0.R loads that and adds many schema, collections of columns, to br0 and writes out many variant tables to ppl0.dat. One of those is *xroadsc*
3. RP subsystem
    1. br1.R is an ad-hoc file, that is unmaintained, and has to be run by hand, it produces ppl1.dat
4. br2.R this loads the imputations into the main dataset. No longer used. These are now stored as CSV.
5. br3.R marks Near-Zero Variance variables and performs the imputation. Produces impute0.dat and xroadsc-3.dat
    1. br3a.R is a helper script for the NZV
    2. br3b.R generates correlatons
6. br4.R is a sanitizer
    1. br4a.R removes specified NZV 
    2. br4a0.R removes prescient variables
    3. br4b.R plots correlations
    4. br4x.R is ad-hoc
7. br5.R and br5a.R deal with over-correlated variables
8. br7.R and br7a.R is the Imbalanced Predictor
    1. br7.R is the split, train, test
    2. br7a.R is the presentation: confusion, ROC
9. br9.R is the SMOTE balanced Predictor xroadsc-9.dat

Note: br8.R is an ad-hoc Random Forest Predictor

In [2]:
load('ppl00.dat', envir=.GlobalEnv)

In [3]:
ls()

[1] "br0"   "ppl00"

In [10]:
br0

$source
[1] "../cache/out/xsamples1.csv"

In [9]:
colnames(ppl00)
head(ppl00)

[1] "smpl0"         "lsoa"          "enq0"          "claim0"       
  [5] "cwy0"          "aid0"          "date0"         "src0"         
  [9] "isclm0"        "ddate0"        "ddate1"        "emethod1"     
 [13] "priority"      "response"      "estatus0"      "month0"       
 [17] "yyyy"          "mm"            "tmax"          "tmin"         
 [21] "af"            "rain"          "sun"           "ssn"          
 [25] "month00"       "yyyy0"         "mm0"           "tmax0"        
 [29] "tmin0"         "af0"           "rain0"         "sun0"         
 [33] "ssn0"          "ncars0"        "ncars1"        "ncars2"       
 [37] "ncars3"        "ncars4"        "ncars00"       "tcars0"       
 [41] "imd"           "imdk"          "wealthk"       "tage0"        
 [45] "tage1"         "tage2"         "tage3"         "tage4"        
 [49] "tage00"        "mage0"         "mage1"         "mage2"        
 [53] "mage3"         "mage4"         "mage00"        "fage0"        
 [57] "fage1"         "fage2"         "fage3"         "fage4"        
 [61] "fage00"        "ln"            "lreds"         "lambers"      
 [65] "lgreens"       "rn"            "rreds"         "rambers"      
 [69] "rgreens"       "distance"      "width"         "area"         
 [73] "isdist"        "pri"           "iscls"         "pri2"         
 [77] "isslip"        "isshared"      "isdual"        "issingle"     
 [81] "isoneway"      "isround"       "lanes"         "lanes2"       
 [85] "rh0"           "mtraffic"      "spdlimit"      "surftype"     
 [89] "siteid"        "isurban"       "nassets"       "a0Xisdist"    
 [93] "a0Xpri"        "a0Xiscls"      "a0Xpri2"       "a0Xisslip"    
 [97] "a0Xisshared"   "a0Xisdual"     "a0Xissingle"   "a0Xisoneway"  
[101] "a0Xisround"    "a0Xlanes"      "a0Xlanes2"     "a0Xrh0"       
[105] "a0Xdistance"   "a0Xwidth"      "a0Xarea"       "a0Xisisolated"
[109] "outcome1"      "mtraffic0"     "distance0"     "nbus"         
[113] "nhospital"     "nhotel"        "nrail"         "nschool"      
[117] "nsupermkt"     "smplbC1"       "smplwtbC1"     "smplriskbC1"  
[121] "ssmplbC1"      "ssmplwtbC1"    "ssmplriskbC1"  "smplfC1"      
[125] "smplwtfC1"     "smplriskfC1"   "ssmplfC1"      "ssmplwtfC1"   
[129] "ssmplriskfC1"  "smplbC2"       "smplwtbC2"     "smplriskbC2"  
[133] "ssmplbC2"      "ssmplwtbC2"    "ssmplriskbC2"  "smplfC2"      
[137] "smplwtfC2"     "smplriskfC2"   "ssmplfC2"      "ssmplwtfC2"   
[141] "ssmplriskfC2"  "statusbC1"     "dfctbC1"       "dfctrfrbC1"   
[145] "dfctinspctbC1" "dfctactionbC1" "dfctcatbC1"    "dfctriskbC1"  
[149] "prmtTbC1"      "prmtbC1"       "prmtwcatbC1"   "prmtddtbC1"   
[153] "prmtriskbC1"   "statusfC1"     "dfctfC1"       "dfctrfrfC1"   
[157] "dfctinspctfC1" "dfctactionfC1" "dfctcatfC1"    "dfctriskfC1"  
[161] "prmtTfC1"      "prmtfC1"       "prmtwcatfC1"   "prmtddtfC1"   
[165] "prmtriskfC1"   "statusbC2"     "dfctbC2"       "dfctrfrbC2"   
[169] "dfctinspctbC2" "dfctactionbC2" "dfctcatbC2"    "dfctriskbC2"  
[173] "prmtTbC2"      "prmtbC2"       "prmtwcatbC2"   "prmtddtbC2"   
[177] "prmtriskbC2"   "statusfC2"     "dfctfC2"       "dfctrfrfC2"   
[181] "dfctinspctfC2" "dfctactionfC2" "dfctcatfC2"    "dfctriskfC2"  
[185] "prmtTfC2"      "prmtfC2"       "prmtwcatfC2"   "prmtddtfC2"   
[189] "prmtriskfC2"

,smpl0,lsoa,enq0,claim0,cwy0,aid0,date0,src0,isclm0,ddate0,⋯,dfctrfrfC2,dfctinspctfC2,dfctactionfC2,dfctcatfC2,dfctriskfC2,prmtTfC2,prmtfC2,prmtwcatfC2,prmtddtfC2,prmtriskfC2
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>,<int>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,E1748235,Welwyn Hatfield 016A,E1748235,NA,0U399/10,0U399,2013-01-01,enq,FALSE,29,⋯,0,0,0,6,0.01,1,0,0,0,0.01
2,E1748146,Welwyn Hatfield 008D,E1748146,NA,0U660/10,0U660,2013-01-01,enq,FALSE,NA,⋯,0,0,0,6,0.01,1,0,0,0,0.01
3,E1748161,Welwyn Hatfield 008D,E1748161,NA,0U660/10,0U660,2013-01-01,enq,FALSE,NA,⋯,0,0,0,6,0.01,1,0,0,0,0.01
4,E1748195,Welwyn Hatfield 008D,E1748195,NA,0U660/10,0U660,2013-01-01,enq,FALSE,NA,⋯,0,0,0,6,0.01,1,0,0,0,0.01
5,E1748207,Welwyn Hatfield 012D,E1748207,NA,0U675/10,0U675,2013-01-01,enq,FALSE,49,⋯,0,0,0,6,0.01,1,0,0,0,0.01
6,E1748208,Welwyn Hatfield 012D,E1748208,NA,0U675/10,0U675,2013-01-01,enq,FALSE,NA,⋯,0,0,0,6,0.01,1,0,0,0,0.01


In [11]:
load('ppl0.dat', envir=.GlobalEnv)

In [12]:
ls()

[1] "br0"       "ppl0"      "ppl00"     "xenqr"     "xlocalr"   "xroadsc"  
[7] "xxwintryr"

In [13]:
dim(xroadsc)

[1] 74418   166

In [14]:
dim(xxwintryr)

[1] 51377   166

In [16]:
br0

$source
[1] "../cache/out/xsamples1.csv"

$seed
[1] 107

$cwy0
 [1] "distance" "width"    "area"     "isdist"   "pri"      "iscls"   
 [7] "pri2"     "isslip"   "isshared" "isdual"   "issingle" "isoneway"
[13] "isround"  "lanes"    "lanes2"   "rh0"      "mtraffic" "spdlimit"
[19] "surftype" "isurban" 

$outcomen
[1] "isclm0"

$outcomen0
[1] "claim"

$cwy1
 [1] "nassets"       "a0Xisdist"     "a0Xpri"        "a0Xiscls"     
 [5] "a0Xpri2"       "a0Xisslip"     "a0Xisshared"   "a0Xisdual"    
 [9] "a0Xissingle"   "a0Xisoneway"   "a0Xisround"    "a0Xlanes"     
[13] "a0Xlanes2"     "a0Xrh0"        "a0Xdistance"   "a0Xwidth"     
[17] "a0Xarea"       "a0Xisisolated"

$cwy2
 [1] "isdist"   "pri"      "iscls"    "pri2"     "isslip"   "isshared"
 [7] "isdual"   "issingle" "isoneway" "isround"  "lanes"    "lanes2"  
[13] "rh0"      "mtraffic" "spdlimit" "surftype" "isurban" 

$cwy3
 [1] "isdist"   "pri"      "iscls"    "pri2"     "isslip"   "isshared"
 [7] "isdual"   "issingle" "isoneway" "isround"  "lanes"    "lanes2"  
[13] "rh0"      "spdlimit" "surftype" "isurban" 

$cwy4
[1] "distance0" "nassets"   "mtraffic0" "isurban"   "local"     "isdist"   

$poi
[1] "nbus"      "nhospital" "nhotel"    "nrail"     "nschool"   "nsupermkt"

$bworks
 [1] "statusbC1"     "dfctbC1"       "dfctrfrbC1"    "dfctinspctbC1"
 [5] "dfctactionbC1" "dfctcatbC1"    "dfctriskbC1"   "prmtTbC1"     
 [9] "prmtbC1"       "prmtwcatbC1"   "prmtddtbC1"    "prmtriskbC1"  
[13] "statusbC2"     "dfctbC2"       "dfctrfrbC2"    "dfctinspctbC2"
[17] "dfctactionbC2" "dfctcatbC2"    "dfctriskbC2"   "prmtTbC2"     
[21] "prmtbC2"       "prmtwcatbC2"   "prmtddtbC2"    "prmtriskbC2"  

$bworks1
[1] "statusbC1"     "dfctbC1"       "dfctrfrbC1"    "dfctinspctbC1"
[5] "dfctactionbC1" "dfctcatbC1"    "dfctriskbC1"  

$fworks
 [1] "statusfC1"     "dfctfC1"       "dfctrfrfC1"    "dfctinspctfC1"
 [5] "dfctactionfC1" "dfctcatfC1"    "dfctriskfC1"   "prmtTfC1"     
 [9] "prmtfC1"       "prmtwcatfC1"   "prmtddtfC1"    "prmtriskfC1"  
[13] "statusfC2"     "dfctfC2"       "dfctrfrfC2"    "dfctinspctfC2"
[17] "dfctactionfC2" "dfctcatfC2"    "dfctriskfC2"   "prmtTfC2"     
[21] "prmtfC2"       "prmtwcatfC2"   "prmtddtfC2"    "prmtriskfC2"  

$weather
 [1] "tmax"  "tmin"  "af"    "rain"  "sun"   "ssn"   "tmax0" "tmin0" "af0"  
[10] "rain0" "sun0"  "ssn0" 

$lsoa
 [1] "ncars0"  "ncars1"  "ncars2"  "ncars3"  "ncars4"  "ncars00" "tcars0" 
 [8] "imd"     "imdk"    "wealthk" "tage0"   "tage1"   "tage2"   "tage3"  
[15] "tage4"   "tage00"  "mage0"   "mage1"   "mage2"   "mage3"   "mage4"  
[22] "mage00"  "fage0"   "fage1"   "fage2"   "fage3"   "fage4"   "fage00" 

$enq0
[1] "emethod1" "priority" "response" "estatus0" "pri"      "pri2"    

$enq1
[1] "emethod1" "priority" "response" "pri"      "pri2"    

$enq2
[1] "emethod1" "response" "pri"      "pri2"     "mtraffic" "rh0"     

$bsmpls
 [1] "smplbC1"      "smplwtbC1"    "smplriskbC1"  "ssmplbC1"     "ssmplwtbC1"  
 [6] "ssmplriskbC1" "smplbC2"      "smplwtbC2"    "smplriskbC2"  "ssmplbC2"    
[11] "ssmplwtbC2"   "ssmplriskbC2"

$fsmpls
 [1] "smplfC1"      "smplwtfC1"    "smplriskfC1"  "ssmplfC1"     "ssmplwtfC1"  
 [6] "ssmplriskfC1" "smplfC2"      "smplwtfC2"    "smplriskfC2"  "ssmplfC2"    
[11] "ssmplwtfC2"   "ssmplriskfC2"

$xroadsc
[1] "ln"      "lreds"   "lambers" "lgreens" "rn"      "rreds"   "rambers"
[8] "rgreens"

$xcols0
[1] "ddate0" "ddate1" "src0"  

$xcols1
[1] "lsoa"   "enq0"   "claim0" "smpl0"  "cwy0"   "siteid"

$xcols2
[1] "yyyy"    "date0"   "month0"  "yyyy0"   "mm0"     "month00"

$xcols3
[1] "aid0"

$prescient
[1] "outcome1" "cost0"    "estatus0" "smplfC1" 

$xlsoar
  [1] "E1749965"  "E1749261"  "E1751025"  "E1753858"  "E1754519"  "E1754863" 
  [7] "E1756007"  "E1758681"  "E1762588"  "E1763525"  "ISTP02676" "ISTP02805"
 [13] "E1765477"  "E1765500"  "ISTP02761" "ISTP02814" "ISTP02922" "ISTP03171"
 [19] "ISTP02546" "ISTP02848" "E1765559"  "E1766005"  "E1766802"  "E1767885" 
 [25] "E1767751"  "E1768150"  "E1768155"  "E1768732